In [ ]:


import pandas as pd
import numpy as np

s2017_df = pd.read_csv('2017_season_data.csv')

players_df = pd.read_csv('player_data.csv')

s2017_df.head(1)
Year	Player	Pos	Age	Tm	G	GS	MP	PER	TS%	...	FT%	ORB	DRB	TRB	AST	STL	BLK	TOV	PF	PTS
0	2017.0	Alex Abrines	SG	23.0	OKC	68.0	6.0	1055.0	10.1	0.56	...	0.898	18.0	68.0	86.0	40.0	37.0	8.0	33.0	114.0	406.0
1 rows × 52 columns

players_df.head(1)
name	year_start	year_end	position	height	weight	birth_date	college
0	Alaa Abdelnaby	1991	1995	F-C	6-10	240.0	June 24, 1968	Duke University
Data Wrangling Activities
1. Merge s2017_df and players_df with a left join

df = s2017_df.merge(players_df, how='left',left_on='Player',right_on='name')

# Use it before modifying the `df` to have a copy
# just in case a modification doesn't go as expected
# df_copy = df.copy()
df.isnull().sum()
Year            0
Player          0
Pos             0
Age             0
Tm              0
G               0
GS              0
MP              0
PER             0
TS%             2
3PAr            2
FTr             2
ORB%            0
DRB%            0
TRB%            0
AST%            0
STL%            0
BLK%            0
TOV%            2
USG%            0
blanl         605
OWS             0
DWS             0
WS              0
WS/48           0
blank2        605
OBPM            0
DBPM            0
BPM             0
VORP            0
FG              0
FGA             0
FG%             2
3P              0
3PA             0
3P%            47
2P              0
2PA             0
2P%             5
eFG%            2
FT              0
FTA             0
FT%            24
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
name            4
year_start      4
year_end        4
position        4
height          4
weight          4
birth_date      4
college       118
dtype: int64
2. Are there misses (mismatches) in the resulting dataframe?

df['name'].isna().sum()
4
3. How many rows couldn't be matched?

s2017_df.iloc[np.where(df.name.isna())].Player
349    Chris McCullough
350    Chris McCullough
352      K.J. McDaniels
593         Paul Zipser
Name: Player, dtype: object
4. Extract the names of the players that couldn't be matched

player_misses = list(df.loc[df.name.isna(),'Player'].values)
5. Modify players_df with the correct names to re-try a successful merge

# Use it before modifying the `df` to have a copy
# just in case a modification doesn't go as expected
df_copy = df.copy()

player_names_replacements = {
     "Luc Mbah a Moute": "Luc Mbah",
     "James Michael McAdoo": "James Michael",
     "Sheldon Mac": "Sheldon McClellan",
     "Metta World Peace": "Metta World",
}
for old_name, new_name in player_names_replacements.items():
    players_df.loc[players_df['name'] == old_name, 'name'] = new_name
6. Perform the merge between s2017_df and players_df again, this time, without misses

df = s2017_df.merge(players_df, how='left', left_on='Player', right_on='name')
​
7. Remove unnecessary columns

columns_to_drop = [
    "Year",
    "PER",
    "TS%",
    "3PAr",
    "FTr",
    "USG%",
    "blanl",
    "OWS",
    "DWS",
    "WS",
    "WS/48",
    "blank2",
    "OBPM",
    "DBPM",
    "BPM",
    "VORP",
    "FG%",
    "3P%",
    "eFG%",
    "FT%",
    "name",
]
df.drop(columns=columns_to_drop,inplace=True)
8. Rename teams to their full name

team_mapping = {
    "OKC": "Oklahoma City Thunder",
    "DAL": "Dallas Mavericks",
    "BRK": "Brooklyn Nets",
    "SAC": "Sacramento Kings",
    "NOP": "New Orleans Pelicans",
    "MIN": "Minnesota Timberwolves",
    "SAS": "San Antonio Spurs",
    "IND": "Indiana Pacers",
    "MEM": "Memphis Grizzlies",
    "POR": "Portland Trail Blazers",
    "CLE": "Cleveland Cavaliers",
    "LAC": "Los Angeles Clippers",
    "PHI": "Philadelphia 76ers",
    "HOU": "Houston Rockets",
    "MIL": "Milwaukee Bucks",
    "NYK": "New York Knicks",
    "DEN": "Denver Nuggets",
    "ORL": "Orlando Magic",
    "MIA": "Miami Heat",
    "PHO": "Phoenix Suns",
    "GSW": "Golden State Warriors",
    "CHO": "Charlotte Hornets",
    "DET": "Detroit Pistons",
    "ATL": "Atlanta Hawks",
    "WAS": "Washington Wizards",
    "LAL": "Los Angeles Lakers",
    "UTA": "Utah Jazz",
    "BOS": "Boston Celtics",
    "CHI": "Chicago Bulls",
    "TOR": "Toronto Raptors"
}
df['Team'] = df.Tm.replace(team_mapping)

​
9. Convert birthday to a datetime object

df['birth_date'] = pd.to_datetime(df.birth_date)
10. Delete all players from the TOT team

# if something goes wrong, just execute this line:
# df = df_copy.copy()

df = df.loc[df.Tm!='TOT']
Analysis
11. What's the team with the most players in the league?

df.groupby('Team').size().idxmax()
'New Orleans Pelicans'
12. What's the team with the lowest FG?

df.groupby('Team')['FG'].sum().idxmin()
'Dallas Mavericks'
13. What's the team with the best FG%?

fg_perc_per_team = df.groupby('Team')[["FG", "FGA"]].sum()
fg_perc_per_team["FG%"] = fg_perc_per_team['FG'] / fg_perc_per_team['FGA']
fg_perc_per_team.sort_values(by="FG%", ascending=False).head()
​
FG	FGA	FG%
Team
Golden State Warriors	3532.0	7140.0	0.494678
San Antonio Spurs	3470.0	7284.0	0.476387
Los Angeles Clippers	3242.0	6819.0	0.475436
Washington Wizards	3388.0	7136.0	0.474776
Milwaukee Bucks	3190.0	6737.0	0.473505
14. What's the difference between the best and worst 3P shooters (by position)?

top_3p_by_position = df.groupby('Pos')[['3P', '3PA']].apply(lambda rows: rows['3P'].sum() / rows['3PA'].sum()
).sort_values(ascending=False)
15. Find the best scorers in each team

df["Best Score per Team"] = df.groupby('Team')['PTS'].transform('max')
best_scorers_per_team = df.loc[
    df['PTS'] == df["Best Score per Team"],
    ['Player', 'Team', 'Pos', 'PTS']
].sort_values(by='PTS', ascending=False)
​
16. Which team has the 'youngest squad', by average player age?

df.groupby('Team')['birth_date'].mean().sort_values(ascending=False)
df['Age in Days'] = (pd.Timestamp.now() - df['birth_date']).dt.days
df.groupby('Team')['Age in Days'].mean().sort_values()
​

Team
Portland Trail Blazers    11618.666667
Toronto Raptors           11951.294118
Boston Celtics            11963.200000
Orlando Magic             12026.315789
Denver Nuggets            12043.263158
Detroit Pistons           12044.266667
Phoenix Suns              12056.888889
Washington Wizards        12080.222222
Charlotte Hornets         12117.631579
Brooklyn Nets             12156.285714
Chicago Bulls             12175.000000
Oklahoma City Thunder     12175.473684
Houston Rockets           12196.722222
Utah Jazz                 12306.200000
Philadelphia 76ers        12338.500000
Miami Heat                12471.200000
Sacramento Kings          12478.684211
New York Knicks           12493.062500
New Orleans Pelicans      12496.814815
Dallas Mavericks          12512.458333
Milwaukee Bucks           12580.350000
Memphis Grizzlies         12618.235294
Minnesota Timberwolves    12767.812500
Indiana Pacers            12904.882353
Golden State Warriors     12909.705882
Los Angeles Lakers        13066.421053
Los Angeles Clippers      13538.600000
Atlanta Hawks             13989.636364
San Antonio Spurs         14184.117647
Cleveland Cavaliers       14246.590909
Name: Age in Days, dtype: float64